In [ ]:
# -*- coding: utf-8 -*-
"""KT.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/11vdsvxqtHO59DiTM8ufTO-KkgeVK3kAm
"""

# Keras-Tuner
!pip install -q -U keras-tuner

# Commented out IPython magic to ensure Python compatibility.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from google.colab import drive

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.models import Sequential
import keras_tuner as kt

drive.mount('/content/drive')
path = '/content/drive/MyDrive/LOBster/MSFT'

os.chdir(path)

df = pd.read_csv("data_msft_ta.csv")
df.dropna(inplace=True)

X = df[['Price', 'OFI']]
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X.values.reshape(X.shape[0], X.shape[1], 1), y, test_size=0.2, stratify=y, random_state=1234)


def model_builder(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('hp_units_1', min_value=4, max_value=128, step=4),
                   activation='relu', input_shape=(X_train.shape[1], 1)))

    for i in range(2, 5):
        model.add(Dropout(0.20))
        model.add(LSTM(units=hp.Int('hp_units_' + str(i),
                                    min_value=4, max_value=128, step=4),
                       activation='relu'))

    model.add(layers.Dense(units=1, activation='linear'))  # Output layer
    # Set the loss function to root mean squared error (RMSE)
    loss = tf.keras.losses.MeanSquaredError()

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
                  loss=loss, metrics=['mape', tf.keras.metrics.RootMeanSquaredError()])

    return model


tuner = kt.Hyperband(model_builder,
                     objective='val_mean_squared_error',
                     max_epochs=50,
                     factor=3,
                     directory='KERAS-TUNER',
                     project_name='LSTM_PRICE_ONLY')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train, y_train,
             batch_size=64,
             epochs=50,
             validation_split=0.2,
             callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.

The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.
""")

print('Best layers units: \n')
for i in range(2, 6):
    print('Layer:', i - 1, ',', best_hps.get('hp_units_' + str(i)), 'units')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

The hyperparameter search is complete.

The optimal learning rate for the optimizer is 0.001.

Best layers units: 

Layer: 1 , 124 units
Layer: 2 , 104 units
Layer: 3 , 128 units
Layer: 4 , 68 units
